In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = 88

In [3]:
# Cambiar ruta a la oficial Google Cloud para no depender del local

df = pd.read_csv("https://storage.googleapis.com/datamexico-data/inegi_intercensal_census/TR_PERSONA32.CSV", 
                 encoding="latin-1",
                 dtype=str,
                 low_memory=False)

In [4]:
# Coordinar columnas con sheets
df.columns = map(str.lower, df.columns)

In [5]:
df.head()

,id_viv,id_persona,ent,nom_ent,mun,nom_mun,loc50k,nom_loc,cobertura,estrato,upm,factor,numper,sexo,edad,parent,parent_otro_c,ident_madre,ident_padre,sersalud,afrodes,acta_nac,dhsersal1,dhsersal2,perte_indigena,ent_pais_nac,nacionalidad,hlengua,qdialect_c,qdialect_inali,hespanol,elengua,asisten,mun_asi,nom_mun_asi,ent_pais_asi,tie_traslado_escu,med_traslado_esc1,med_traslado_esc2,med_traslado_esc3,escolari,nivacad,alfabet,escoacum,mun_res10,nom_mun_res10,ent_pais_res10,situa_conyugal,ident_pareja,conact,ocupacion_c,situacion_trab,aguinaldo,vacaciones,servicio_medico,utilidades,incap_sueldo,sar_afore,credito_vivienda,ingtrmen,actividades_c,mun_trab,nom_mun_trab,ent_pais_trab,tie_traslado_trab,med_traslado_trab1,med_traslado_trab2,med_traslado_trab3,acti_sin_pago1,acti_sin_pago2,acti_sin_pago3,acti_sin_pago4,acti_sin_pago5,acti_sin_pago6,acti_sin_pago7,acti_sin_pago8,hijos_nac_vivos,hijos_fallecidos,hijos_sobreviv,fecha_nac_m,fecha_nac_a,sobrevivencia,edad_morir_d,edad_morir_m,edad_morir_a,tamloc
0,320010000001,32001000000102,32,Zacatecas,001,Apozol,0000,Localidad de menos de 50 mil habitantes,2,32-001-04,284040,1,2,3,58,2,201,97,97,5,3,1,8,NaN,3,032,5,3,NaN,NaN,NaN,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,7,1,001,Apozol,032,5,1,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,2,2,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,320010000001,32001000000101,32,Zacatecas,001,Apozol,0000,Localidad de menos de 50 mil habitantes,2,32-001-04,284040,1,1,1,62,1,101,96,96,1,3,1,8,NaN,3,032,5,3,NaN,NaN,NaN,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,5,6,001,Apozol,032,5,2,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,320010000002,32001000000207,32,Zacatecas,001,Apozol,0000,Localidad de menos de 50 mil habitantes,2,32-001-04,284052,1,7,1,2,3,301,2,1,6,3,1,8,NaN,1,032,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,320010000002,32001000000205,32,Zacatecas,001,Apozol,0000,Localidad de menos de 50 mil habitantes,2,32-001-04,284052,1,5,1,10,3,301,2,1,6,3,1,8,NaN,1,032,5,3,NaN,NaN,NaN,3,5,001,Apozol,032,1,6,NaN,NaN,4,2,5,4,001,Apozol,032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,320010000002,32001000000203,32,Zacatecas,001,Apozol,0000,Localidad de menos de 50 mil habitantes,2,32-001-04,284052,1,3,3,17,3,301,2,1,6,3,1,8,NaN,1,032,5,3,NaN,NaN,NaN,3,5,001,Apozol,032,1,6,NaN,NaN,1,4,5,10,001,Apozol,032,6,NaN,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [6]:
# Agregar columnas de ids y rename de factor a population
df["mun_id"] = df["ent"] + df["mun"]
df["loc_id"] = df["mun_id"] + df["loc50k"]
df.rename(index=str, columns={"factor":"population"}, inplace=True)

In [7]:
# Convertir elementos necesarios a int
df["mun_id"] = df["mun_id"].astype(int)
df["loc_id"] = df["loc_id"].astype(int)
df["population"] = df["population"].astype(int)

In [8]:
# Separar columnas en 3 listas: 
                                # values, columnas de enteros
                                # params, parametros a trabajar
                                # sides, columnas en el spreedsheet no agregadas todavia

values = [
    "mun_id",
    "loc_id",
    "population"
]

params = [
    "sexo",
    "parent",
    "sersalud",
    "dhsersal1",
    "nacionalidad"
]

sides = [
    "ident_madre",
    "ident_padre",
    "tamloc",
    "conact"
]

In [9]:
# Cargar spreedsheet para realizar los cambios a la tabla de Personas
df_labels = pd.ExcelFile("https://docs.google.com/spreadsheets/d/e/2PACX-1vR08Js9Sh4nNTMe5uBcsDUFedG5MOjIf90p6EHAr1_sWY5kpnI3xUvyPHzQpTEUrXz1pskaoc0uyea6/pub?output=xlsx")

for sheet in params:
    df_l = pd.read_excel(df_labels, sheet)
    print(sheet)
    df[sheet] = df[sheet].astype(int)
    df[sheet] = df[sheet].replace(dict(zip(df_l.prev_id, df_l.id)))
 

sexo
parent
sersalud
dhsersal1
nacionalidad


In [10]:
df[params].head()

,sexo,parent,sersalud,dhsersal1,nacionalidad
0,2,2,5,8,1
1,1,1,1,8,1
2,1,3,6,8,1
3,1,3,6,8,1
4,2,3,6,8,1


In [11]:
df = df.groupby(params + ["mun_id", "loc_id"]).sum().reset_index()

In [12]:
df

,sexo,parent,sersalud,dhsersal1,nacionalidad,mun_id,loc_id,population
0,1,0,0,0,0,32007,320070000,4
1,1,0,0,0,0,32008,320080000,1
2,1,0,0,0,0,32010,320100000,6
3,1,0,0,0,0,32011,320110000,1
4,1,0,0,0,0,32012,320120000,2
5,1,0,0,0,0,32016,320160000,3
6,1,0,0,0,0,32017,320170001,1
7,1,0,0,0,0,32022,320220000,9
8,1,0,0,0,0,32023,320230000,2
9,1,0,0,0,0,32024,320240000,16
